In [1]:
import os, glob
import numpy as np
import pandas as pd
import math
import pickle
import matplotlib.pyplot as plt
#from scipy.stats import norm
# from scipy import stats
# from statsmodels.stats.weightstats import ztest
from IPython.display import display, HTML

In [2]:
non_coding_region="core_prom"
core_prom_type= "TATA"
cancer_type= "Brain"

In [3]:
base_dir = "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/DNABERT_Data/{}/Patient_wise".format(cancer_type, non_coding_region)
output_dir= "/home/pdutta/Data/Cancer_wiseGDC/New_data/{}/Generated_files/Variant_Analysis/{}/".format(cancer_type, non_coding_region)
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [4]:
df_core_prom = pd.read_csv("/home/pdutta/Data/Noncoding_region/core_prom_bed.tsv", sep="\t")
df_core_prom

,Chr,gene_id,transcript_id,exon_id,strand,TSS,start,end
0,chr7,ENSG00000004059.11,ENST00000000233.10,ENSE00001872691.2,+,127588411,127588366,127588455
1,chr7,ENSG00000001626.16,ENST00000003084.11,ENSE00001343851.2,+,117480025,117479980,117480069
2,chr7,ENSG00000001630.17,ENST00000003100.13,ENSE00001216550.10,-,92134477,92134433,92134522
3,chr7,ENSG00000002933.9,ENST00000004103.8,ENSE00001827596.2,+,150800769,150800724,150800813
4,chr7,ENSG00000004799.8,ENST00000005178.6,ENSE00001133344.6,-,95596516,95596472,95596561
...,...,...,...,...,...,...,...,...
236970,chr15,ENSG00000103876.14,ENST00000684569.1,ENSE00003921718.1,+,80153010,80152965,80153054
236971,chr15,ENSG00000213614.11,ENST00000684602.1,ENSE00002620879.1,-,72375989,72375945,72376034
236972,chr15,ENSG00000213614.11,ENST00000684667.1,ENSE00002584031.1,-,72375972,72375928,72376017
236973,chr15,ENSG00000104133.16,ENST00000684676.1,ENSE00003922470.1,-,44663688,44663644,44663733


In [5]:
with open("/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/Intersected_Data/intersected_vcf_{}_data.pkl".format(non_coding_region), "rb") as file:
    loaded_dictionary = pickle.load(file)

In [6]:
len(loaded_dictionary)

432

In [7]:
def log_oods_ratio(p1,p2):
    term1 = math.log2(p1/(1-p1))
    term2 = math.log2(p2/(1-p2))
    return (term1-term2)
def calculate_diff_probab(p1,p2):
    return((p2-p1)*max(p1,p2))

In [8]:
patient_dfs= []
patient_id = []
for dirpath, dirnames, filenames in os.walk(base_dir):
    #print(dirpath, dirnames, filenames)
    for dir_name in dirnames:
        dir_path = dirpath +"/" + dir_name
        print(dir_path)
        patient_id.append(dir_name)
        print(dir_name)
        #input()
        df_temp = loaded_dictionary[dir_name]
        print(len(df_temp))
        df_temp = df_temp.drop_duplicates(subset=['chr_name', 'start', 'end', 'transcript_id','START_POS', 'END_POS', 'REF','ALT']).reset_index()
        concat_probab = np.load(dir_path+"/{}/Prediction_result/pred_results.npy".format(core_prom_type))
        print(len(concat_probab))
        print(len(df_temp))
        if(2*len(df_temp)==len(concat_probab)):
            #input()
            Reference_probab = concat_probab[::2]
            Alternative_probab = concat_probab[1::2]
            df_temp['Ref_probab']=Reference_probab
            df_temp['Alt_probab']=Alternative_probab
            df_temp['ScoreChange'] = df_temp.apply(lambda row: calculate_diff_probab(row['Ref_probab'], row['Alt_probab']), axis=1)
            df_temp['LogOddRatio'] = df_temp.apply(lambda row: log_oods_ratio(row['Ref_probab'], row['Alt_probab']), axis=1)
            #display(df_temp)
            #input()
            # Apply the filter conditions
            df_temp = df_temp[((df_temp['Ref_probab'] > 0.5) & (df_temp['Alt_probab'] <= 0.5)) | 
                      ((df_temp['Ref_probab'] <= 0.5) & (df_temp['Alt_probab'] > 0.5))]
            #display(df_temp)
            #input()
            patient_dfs.append(df_temp)
        else:
            print(dir_name)
            input()
        #input()
    break

/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel
a5867fd1-4897-4559-855a-c30002d3c6bb_sanger_raw_pindel
4395
8790
4395
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel
0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_raw_pindel
4155
8310
4155
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan
3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan
4282
8564
4282
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel
af56a9e4-b896-4d47-b9c0-930eb68862bd_sanger_raw_pindel
4419
8838
4419
/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/67c83385-8497-4c2d-9

FileNotFoundError: [Errno 2] No such file or directory: '/home/pdutta/Data/Cancer_wiseGDC/New_data/Brain/Generated_files/DNABERT_Data/core_prom/Patient_wise/4132ab70-457f-41a9-aae3-340f9fe109e2_sanger_raw_pindel/TATA/Prediction_result/pred_results.npy'

In [9]:
patient_dfs[0]

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,PC,RE,REP,RS,S1,S2,Ref_probab,Alt_probab,ScoreChange,LogOddRatio
810,1065,chr12,7620216,7620305,ENSG00000284327.1,ENST00000639256.1,ENSE00003807025.1,-,7620260,chr12,...,D,7620277,1,7620271,8,340.33599853515625,0.000020,0.999901,0.999782,-28.906130
1098,1400,chr1,150153438,150153527,ENSG00000023902.14,ENST00000607609.2,ENSE00003699801.2,+,150153483,chr1,...,I,150153504,10,150153492,84,1407.1700439453125,0.999990,0.000212,-0.999767,28.745356
1184,1494,chr1,89589498,89589587,ENSG00000231999.7,ENST00000660444.1,ENSE00003856686.1,-,89589542,chr1,...,I,89589573,13,89589558,162,2050.280029296875,0.000071,0.999988,0.999906,-30.166860
1365,1749,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,...,I,91159595,4,91159572,8,574.5590209960938,0.999987,0.000221,-0.999754,28.403391
2566,3288,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,D,13722235,26,13722208,8,573.4400024414062,0.243720,0.830184,0.486873,-3.923158
2661,3415,chr10,106140219,106140308,ENSG00000229775.7,ENST00000449761.5,ENSE00001724579.1,+,106140264,chr10,...,D,106140226,2,106140223,130,1784.489990234375,0.999980,0.003048,-0.996912,23.969668
2701,3467,chr10,79968168,79968257,ENSG00000273372.1,ENST00000608229.1,ENSE00003708259.1,+,79968213,chr10,...,I,79968235,1,79968232,72,1261.2099609375,0.000037,0.999781,0.999525,-26.861017
3634,4653,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,I,23897327,17,23897308,28,682.8259887695312,0.125368,0.999989,0.874612,-19.326757
3635,4654,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,D,23897327,18,23897308,7,497.3609924316406,0.125368,0.997924,0.870745,-11.711714
3850,4959,chr17,36001577,36001666,ENSG00000275688.4,ENST00000616694.1,ENSE00003720771.1,-,36001621,chr17,...,I,36001618,0,36001607,56,1051.9300537109375,0.999994,0.000257,-0.999731,29.279976


In [10]:
# Concatenate the patient dataframes and drop duplicates to get all unique regions across the patients
unique_patient_regions = pd.concat(patient_dfs).drop_duplicates(subset=['chr_name', 'transcript_id', 'start', 'end', 'START_POS', 'END_POS', 'REF', 'ALT'])
unique_patient_regions

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
810,1065,chr12,7620216,7620305,ENSG00000284327.1,ENST00000639256.1,ENSE00003807025.1,-,7620260,chr12,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1098,1400,chr1,150153438,150153527,ENSG00000023902.14,ENST00000607609.2,ENSE00003699801.2,+,150153483,chr1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1184,1494,chr1,89589498,89589587,ENSG00000231999.7,ENST00000660444.1,ENSE00003856686.1,-,89589542,chr1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1365,1749,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2566,3288,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3789,4974,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1912,2525,chr4,143555358,143555447,ENSG00000248924.2,ENST00000515019.2,ENSE00002063552.2,-,143555402,chr4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1717,2236,chr4,141133418,141133507,ENSG00000170153.11,ENST00000306799.7,ENSE00001133687.2,-,141133462,chr4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1811,2360,chr4,141133425,141133514,ENSG00000170153.11,ENST00000515673.7,ENSE00003847744.1,-,141133469,chr4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
filtered_regions = unique_patient_regions.groupby(['chr_name', 'transcript_id', 'start', 'end']).filter(lambda x: len(x) > 1 and not all(x['START_POS'].eq(x['START_POS'].iloc[0]) & x['END_POS'].eq(x['END_POS'].iloc[0]) & x['REF'].eq(x['REF'].iloc[0]) & x['ALT'].eq(x['ALT'].iloc[0])))
filtered_regions

,index,chr_name,start,end,gene_id,transcript_id,exon_id,strand,TSS,CHROM,...,CLPM,DP,DS,GP,MP,SG,SNP,SP,TG,TP
1365,1749,chr9,91159528,91159617,ENSG00000235641.5,ENST00000423719.3,ENSE00003702836.1,+,91159573,chr9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2566,3288,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3634,4653,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3635,4654,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2616,2616,chrX,13722196,13722285,ENSG00000196459.15,ENST00000380578.1,ENSE00001485528.1,-,13722240,chrX,...,0.0,52.0,.,0.042,0.95,AA/AAACC,False,0.072,AA/AAAAC,0.88
3075,4362,chr5,83581276,83581365,ENSG00000249835.2,ENST00000513899.1,ENSE00002036499.1,-,83581320,chr5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3076,4363,chr5,83581276,83581365,ENSG00000249835.2,ENST00000513899.1,ENSE00002036499.1,-,83581320,chr5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3712,5206,chr10,78038511,78038600,ENSG00000138326.21,ENST00000480662.2,ENSE00001880601.2,+,78038556,chr10,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5059,7021,chr19,23897276,23897365,ENSG00000287399.1,ENST00000658651.1,ENSE00003857817.1,+,23897321,chr19,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2551,3310,chr2,229923195,229923284,ENSG00000153827.14,ENST00000435716.5,ENSE00001725866.1,-,229923239,chr2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
pd.set_option('display.max_columns', None)
filtered_regions[filtered_regions['start']==47977248]

In [12]:
# Reset the option to its default (typically 20)
pd.reset_option('display.max_columns')

In [ ]:
unique_patient_regions[unique_patient_regions['start']==47977248]

In [13]:
# Step 3: Create a new dataframe with three rows for each patient, with columns as the acceptor coordinates
# Initialize the dictionaries
data_ref, data_alt, data_log_odd, data_score_change = {}, {}, {}, {}

# Loop over each region in the unique_patient_regions
for _, region in unique_patient_regions.iterrows():
    key = f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'
    data_ref[key] = []
    data_alt[key] = []
    data_log_odd[key] = []
    data_score_change[key] = []

# Loop through each patient and for each region, get the Alt_probab score for that region for that patient
for df in patient_dfs:
    print(df.shape)
    #print(df)
    for _, region in unique_patient_regions.iterrows():
        #print(region)
        #input()
        for data, score_column in zip([data_ref, data_alt, data_log_odd, data_score_change], ['Ref_probab', 'Alt_probab', 'LogOddRatio', 'ScoreChange']):
            score_series = df.loc[(df['chr_name'] == region.chr_name) & (df['start'] == region.start)& (df['end'] == region.end) & (df['START_POS'] == region.START_POS) & (df['END_POS'] == region.END_POS) & (df['REF'] == region.REF) & (df['ALT'] == region.ALT) , score_column]
            #print(score_series)
            if score_series.empty:
                score = np.nan
            else:
                score = score_series.max()
                #print(score_series , score)
                #input()
            data[f'{region.chr_name}_{region.start}_{region.end}_{region.START_POS}_{region.END_POS}_{region.REF}_{region.ALT}'].append(score)

(10, 30)
(1, 33)
(15, 30)
(1, 33)
(8, 30)
(0, 33)
(8, 30)
(0, 33)
(3, 33)
(0, 33)
(0, 33)
(6, 30)
(8, 30)
(10, 30)
(0, 33)
(6, 30)
(12, 30)
(10, 30)
(8, 30)
(0, 33)
(1, 33)
(8, 30)
(4, 30)
(7, 30)
(11, 30)
(0, 33)
(0, 33)
(0, 33)
(5, 30)
(8, 30)
(9, 30)
(7, 30)
(10, 30)
(9, 30)
(0, 33)
(1, 33)
(0, 33)
(7, 30)
(0, 33)
(0, 33)
(8, 30)
(6, 30)
(6, 30)
(9, 30)
(0, 33)
(0, 33)
(2, 33)
(8, 30)
(9, 30)
(0, 33)
(8, 30)
(0, 33)
(7, 30)
(0, 33)
(0, 33)
(10, 30)
(6, 30)
(7, 30)
(0, 33)
(8, 30)
(0, 33)
(0, 33)
(0, 33)
(8, 30)
(0, 33)
(0, 33)
(0, 33)
(7, 30)
(0, 33)
(10, 30)
(0, 33)
(5, 30)
(0, 33)
(0, 33)
(0, 33)
(1, 33)
(10, 30)
(0, 33)
(7, 30)
(10, 30)
(0, 33)
(0, 33)
(0, 33)
(0, 33)
(6, 30)
(10, 30)
(6, 30)
(12, 30)
(0, 33)
(0, 33)
(1, 33)
(14, 30)
(9, 30)
(5, 30)
(0, 33)
(9, 30)
(0, 33)
(11, 30)
(8, 30)
(10, 30)
(0, 33)
(0, 33)
(0, 33)
(0, 33)
(0, 33)
(0, 33)
(8, 30)
(0, 33)
(10, 30)
(12, 30)


In [14]:
result_df_ref = pd.DataFrame(data_ref)
result_df_ref.insert(0, "Patient_ID", patient_id)
result_df_ref

,Patient_ID,chr12_7620216_7620305_7620270_7620275_ATAAT_A,chr1_150153438_150153527_150153491_150153492_A_AT,chr1_89589498_89589587_89589557_89589558_C_CT,chr9_91159528_91159617_91159571_91159572_G_GAGGA,chrX_13722196_13722285_13722207_13722210_CAA_C,chr10_106140219_106140308_106140222_106140224_GA_G,chr10_79968168_79968257_79968231_79968232_G_GT,chr19_23897276_23897365_23897307_23897308_A_ATTTT,chr19_23897276_23897365_23897307_23897309_AT_A,...,chr12_126927707_126927796_126927728_126927729_A_AAAAG,chr7_126533861_126533950_126533932_126533933_A_G,chr4_141133418_141133507_141133471_141133478_TTGTGTG_T,chr3_47443258_47443347_47443271_47443278_ACACACT_A,chr3_142558957_142559046_142558985_142558989_AAAG_A,chr19_23897276_23897365_23897325_23897327_TG_T,chr4_143555358_143555447_143555376_143555380_CCAT_C,chr4_141133418_141133507_141133471_141133472_T_TTGTG,chr4_141133425_141133514_141133471_141133472_T_TTGTG,chr10_78038511_78038600_78038538_78038542_CTTT_C
0,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.00002,0.99999,0.000071,0.999987,0.24372,0.99998,0.000037,0.125368,0.125368,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_ra...,NaN,0.99999,0.000071,0.999987,NaN,0.99998,0.000037,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,535e0e9e-9757-45b3-aafd-2b80908de134_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9a3edc2a-0eb3-4d88-965d-2801cd19df51_sanger_ra...,NaN,0.99999,0.000071,0.999987,NaN,0.99998,0.000037,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,90c2456b-af8b-4e31-ae86-7de05a0f4124_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,f8b8e0df-4234-4839-b0ee-34a2c9ef4985_sanger_ra...,NaN,0.99999,NaN,NaN,NaN,0.99998,0.000037,NaN,0.125368,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000053,0.999975,NaN
107,7bed15a4-3b1e-487e-954d-e6b55e296dc0_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,ca52ea0e-5021-463b-8821-5e7c765812ff_sanger_ra...,NaN,0.99999,NaN,NaN,NaN,0.99998,0.000037,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.857474


In [15]:
result_df_alt = pd.DataFrame(data_alt)
result_df_alt.insert(0, "Patient_ID", patient_id)
result_df_alt

,Patient_ID,chr12_7620216_7620305_7620270_7620275_ATAAT_A,chr1_150153438_150153527_150153491_150153492_A_AT,chr1_89589498_89589587_89589557_89589558_C_CT,chr9_91159528_91159617_91159571_91159572_G_GAGGA,chrX_13722196_13722285_13722207_13722210_CAA_C,chr10_106140219_106140308_106140222_106140224_GA_G,chr10_79968168_79968257_79968231_79968232_G_GT,chr19_23897276_23897365_23897307_23897308_A_ATTTT,chr19_23897276_23897365_23897307_23897309_AT_A,...,chr12_126927707_126927796_126927728_126927729_A_AAAAG,chr7_126533861_126533950_126533932_126533933_A_G,chr4_141133418_141133507_141133471_141133478_TTGTGTG_T,chr3_47443258_47443347_47443271_47443278_ACACACT_A,chr3_142558957_142559046_142558985_142558989_AAAG_A,chr19_23897276_23897365_23897325_23897327_TG_T,chr4_143555358_143555447_143555376_143555380_CCAT_C,chr4_141133418_141133507_141133471_141133472_T_TTGTG,chr4_141133425_141133514_141133471_141133472_T_TTGTG,chr10_78038511_78038600_78038538_78038542_CTTT_C
0,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.999901,0.000212,0.999988,0.000221,0.830184,0.003048,0.999781,0.999989,0.997924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_ra...,NaN,0.000212,0.999988,0.000221,NaN,0.003048,0.999781,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,535e0e9e-9757-45b3-aafd-2b80908de134_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9a3edc2a-0eb3-4d88-965d-2801cd19df51_sanger_ra...,NaN,0.000212,0.999988,0.000221,NaN,0.003048,0.999781,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,90c2456b-af8b-4e31-ae86-7de05a0f4124_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,f8b8e0df-4234-4839-b0ee-34a2c9ef4985_sanger_ra...,NaN,0.000212,NaN,NaN,NaN,0.003048,0.999781,NaN,0.997924,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999991,0.00021,NaN
107,7bed15a4-3b1e-487e-954d-e6b55e296dc0_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,ca52ea0e-5021-463b-8821-5e7c765812ff_sanger_ra...,NaN,0.000212,NaN,NaN,NaN,0.003048,0.999781,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000175


In [16]:
result_df_score_change = pd.DataFrame(data_score_change)
result_df_score_change.insert(0, "Patient_ID", patient_id)
result_df_score_change

,Patient_ID,chr12_7620216_7620305_7620270_7620275_ATAAT_A,chr1_150153438_150153527_150153491_150153492_A_AT,chr1_89589498_89589587_89589557_89589558_C_CT,chr9_91159528_91159617_91159571_91159572_G_GAGGA,chrX_13722196_13722285_13722207_13722210_CAA_C,chr10_106140219_106140308_106140222_106140224_GA_G,chr10_79968168_79968257_79968231_79968232_G_GT,chr19_23897276_23897365_23897307_23897308_A_ATTTT,chr19_23897276_23897365_23897307_23897309_AT_A,...,chr12_126927707_126927796_126927728_126927729_A_AAAAG,chr7_126533861_126533950_126533932_126533933_A_G,chr4_141133418_141133507_141133471_141133478_TTGTGTG_T,chr3_47443258_47443347_47443271_47443278_ACACACT_A,chr3_142558957_142559046_142558985_142558989_AAAG_A,chr19_23897276_23897365_23897325_23897327_TG_T,chr4_143555358_143555447_143555376_143555380_CCAT_C,chr4_141133418_141133507_141133471_141133472_T_TTGTG,chr4_141133425_141133514_141133471_141133472_T_TTGTG,chr10_78038511_78038600_78038538_78038542_CTTT_C
0,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,0.999782,-0.999767,0.999906,-0.999754,0.486873,-0.996912,0.999525,0.874612,0.870745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_ra...,NaN,-0.999767,0.999906,-0.999754,NaN,-0.996912,0.999525,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,535e0e9e-9757-45b3-aafd-2b80908de134_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9a3edc2a-0eb3-4d88-965d-2801cd19df51_sanger_ra...,NaN,-0.999767,0.999906,-0.999754,NaN,-0.996912,0.999525,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,90c2456b-af8b-4e31-ae86-7de05a0f4124_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,f8b8e0df-4234-4839-b0ee-34a2c9ef4985_sanger_ra...,NaN,-0.999767,NaN,NaN,NaN,-0.996912,0.999525,NaN,0.870745,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.999929,-0.999739,NaN
107,7bed15a4-3b1e-487e-954d-e6b55e296dc0_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,ca52ea0e-5021-463b-8821-5e7c765812ff_sanger_ra...,NaN,-0.999767,NaN,NaN,NaN,-0.996912,0.999525,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.735112


In [17]:
result_df_log_odd = pd.DataFrame(data_log_odd)
result_df_log_odd.insert(0, "Patient_ID", patient_id)
result_df_log_odd

,Patient_ID,chr12_7620216_7620305_7620270_7620275_ATAAT_A,chr1_150153438_150153527_150153491_150153492_A_AT,chr1_89589498_89589587_89589557_89589558_C_CT,chr9_91159528_91159617_91159571_91159572_G_GAGGA,chrX_13722196_13722285_13722207_13722210_CAA_C,chr10_106140219_106140308_106140222_106140224_GA_G,chr10_79968168_79968257_79968231_79968232_G_GT,chr19_23897276_23897365_23897307_23897308_A_ATTTT,chr19_23897276_23897365_23897307_23897309_AT_A,...,chr12_126927707_126927796_126927728_126927729_A_AAAAG,chr7_126533861_126533950_126533932_126533933_A_G,chr4_141133418_141133507_141133471_141133478_TTGTGTG_T,chr3_47443258_47443347_47443271_47443278_ACACACT_A,chr3_142558957_142559046_142558985_142558989_AAAG_A,chr19_23897276_23897365_23897325_23897327_TG_T,chr4_143555358_143555447_143555376_143555380_CCAT_C,chr4_141133418_141133507_141133471_141133472_T_TTGTG,chr4_141133425_141133514_141133471_141133472_T_TTGTG,chr10_78038511_78038600_78038538_78038542_CTTT_C
0,0f158d5f-16c7-4fb7-b48c-0ff392f1706b_sanger_ra...,-28.90613,28.745356,-30.16686,28.403391,-3.923158,23.969668,-26.861017,-19.326757,-11.711714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3adc47fa-b27a-4df1-98ab-201ebab3c6bf_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ef6c3ecc-0bb6-4035-9c45-57740f5bcaa1_sanger_ra...,NaN,28.745356,-30.16686,28.403391,NaN,23.969668,-26.861017,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,535e0e9e-9757-45b3-aafd-2b80908de134_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9a3edc2a-0eb3-4d88-965d-2801cd19df51_sanger_ra...,NaN,28.745356,-30.16686,28.403391,NaN,23.969668,-26.861017,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,90c2456b-af8b-4e31-ae86-7de05a0f4124_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106,f8b8e0df-4234-4839-b0ee-34a2c9ef4985_sanger_ra...,NaN,28.745356,NaN,NaN,NaN,23.969668,-26.861017,NaN,-11.711714,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-30.995852,27.4934,NaN
107,7bed15a4-3b1e-487e-954d-e6b55e296dc0_CaVEMan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
108,ca52ea0e-5021-463b-8821-5e7c765812ff_sanger_ra...,NaN,28.745356,NaN,NaN,NaN,23.969668,-26.861017,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,15.071915


In [18]:
result_df_ref.to_csv(output_dir+"df_ref_score.tsv", sep="\t", index=False)
result_df_alt.to_csv(output_dir+"df_alt_score.tsv", sep="\t", index=False)
result_df_log_odd.to_csv(output_dir+"df_log_odd_score.tsv", sep="\t", index=False)
result_df_score_change.to_csv(output_dir+"df_score_change.tsv", sep="\t", index=False)